"모든 참가자의 '제출'을 목표로 합니다"를 기반으로 하여 만든 klue/roberta를 이용한 베이스라인입니다.



pytorch를 처음 접하시는 분들을 위해 라인마다 아는대로 설명을 써놨으나



저도 배운지 얼마 안되어서 부족한 면도 있으니 양해부탁드립니다.

우선 필요한 패키지들을 import합니다. 처음 보실법한 패키지들을 설명드리면



1. transformers : Huggingface에 등록된 pretrained model/tokenizer를 불러올 수 있는 패키지

2. torch : 머신러닝 패키지

3. tqdm : batch별로 프로세스할 때 진행상황을 바로 보여주는 패키지



나머지는 아실거라 짐작하고 넘어가겠습니다.

In [1]:
# from google.colab import drive

# drive.mount('/content/drive')

# import sys

# sys.path.append('/content/drive/MyDrive/project_klue')  # eda_nlp 경로를 추가

# %cd /content/drive/MyDrive/project_klue

# %pwd

ModuleNotFoundError: No module named 'google.colab'

In [1]:
# https://dacon.io/en/competitions/official/236037/overview/description



import pandas as pd

import numpy as np

import torch

import os

import random

from sklearn.model_selection import train_test_split

from transformers import AutoModel, AutoTokenizer

from torch.utils.data import Dataset

from torch.utils.data import DataLoader

from torch import nn

from tqdm import tqdm

# from eda import *



# for graphing

import seaborn as sns

import matplotlib.pyplot as plt

우선 데이터를 불러와야겠죠?



예측하는데 ID는 필요없기 때문에 drop 해줬습니다.

In [3]:
train_original = pd.read_csv('/kaggle/input/kluedata/train.csv')

train_original.drop(columns=['ID'], inplace=True)

test = pd.read_csv('/kaggle/input/kluedata/test.csv')

test.drop(columns=['ID'], inplace=True)

submission = pd.read_csv('/kaggle/input/kluedata/sample_submission.csv')

train_original.shape, test.shape, submission.shape

((16541, 6), (7090, 1), (7090, 2))

In [4]:
# import random

# import pickle

# import re



# wordnet = {}

# with open("wordnet.pickle", "rb") as f:

# 	wordnet = pickle.load(f)





# # 한글만 남기고 나머지는 삭제

# def get_only_hangul(line):

# 	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)



# 	return parseText







# ########################################################################

# # Synonym replacement

# # Replace n words in the sentence with synonyms from wordnet

# ########################################################################

# def synonym_replacement(words, n):

# 	new_words = words.copy()

# 	random_word_list = list(set([word for word in words]))

# 	random.shuffle(random_word_list)

# 	num_replaced = 0

# 	for random_word in random_word_list:

# 		synonyms = get_synonyms(random_word)

# 		if len(synonyms) >= 1:

# 			synonym = random.choice(list(synonyms))

# 			new_words = [synonym if word == random_word else word for word in new_words]

# 			num_replaced += 1

# 		if num_replaced >= n:

# 			break



# 	if len(new_words) != 0:

# 		sentence = ' '.join(new_words)

# 		new_words = sentence.split(" ")



# 	else:

# 		new_words = ""



# 	return new_words





# def get_synonyms(word):

# 	synomyms = []



# 	try:

# 		for syn in wordnet[word]:

# 			for s in syn:

# 				synomyms.append(s)

# 	except:

# 		pass



# 	return synomyms



# ########################################################################

# # Random deletion

# # Randomly delete words from the sentence with probability p

# ########################################################################

# def random_deletion(words, p):

# 	if len(words) == 1:

# 		return words



# 	new_words = []

# 	for word in words:

# 		r = random.uniform(0, 1)

# 		if r > p:

# 			new_words.append(word)



# 	if len(new_words) == 0:

# 		rand_int = random.randint(0, len(words)-1)

# 		return [words[rand_int]]



# 	return new_words



# ########################################################################

# # Random swap

# # Randomly swap two words in the sentence n times

# ########################################################################

# def random_swap(words, n):

# 	new_words = words.copy()

# 	for _ in range(n):

# 		new_words = swap_word(new_words)



# 	return new_words



# def swap_word(new_words):

# 	random_idx_1 = random.randint(0, len(new_words)-1)

# 	random_idx_2 = random_idx_1

# 	counter = 0



# 	while random_idx_2 == random_idx_1:

# 		random_idx_2 = random.randint(0, len(new_words)-1)

# 		counter += 1

# 		if counter > 3:

# 			return new_words



# 	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]

# 	return new_words



# ########################################################################

# # Random insertion

# # Randomly insert n words into the sentence

# ########################################################################

# def random_insertion(words, n):

# 	new_words = words.copy()

# 	for _ in range(n):

# 		add_word(new_words)



# 	return new_words





# def add_word(new_words):

# 	synonyms = []

# 	counter = 0

# 	while len(synonyms) < 1:

# 		if len(new_words) >= 1:

# 			random_word = new_words[random.randint(0, len(new_words)-1)]

# 			synonyms = get_synonyms(random_word)

# 			counter += 1

# 		else:

# 			random_word = ""



# 		if counter >= 10:

# 			return



# 	random_synonym = synonyms[0]

# 	random_idx = random.randint(0, len(new_words)-1)

# 	new_words.insert(random_idx, random_synonym)







# def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):

#     print('EDA******************************************')

#     sentence = get_only_hangul(sentence)

#     words = sentence.split(' ')

#     words = [word for word in words if word != ""]

#     num_words = len(words)

#     print('word', '!'*50)

#     print(words)

#     augmented_sentences = []

#     num_new_per_technique = int(num_aug/4) + 1



#     n_sr = max(1, int(alpha_sr*num_words))

#     n_ri = max(1, int(alpha_ri*num_words))

#     n_rs = max(1, int(alpha_rs*num_words))



#     # sr

#     for _ in range(num_new_per_technique):

#         print('sr'*10)

#         a_words = synonym_replacement(words, n_sr)

#         print('sr words', a_words)

#         augmented_sentences.append(' '.join(a_words))



#     # ri

#     for _ in range(num_new_per_technique):

#         print('ri'*10)



#         a_words = random_insertion(words, n_ri)

#         print('ri words', a_words)

#         augmented_sentences.append(' '.join(a_words))



#     # rs

#     for _ in range(num_new_per_technique):

#         print('rs'*10)

#         a_words = random_swap(words, n_rs)

#         print('rs words', a_words)

#         augmented_sentences.append(" ".join(a_words))



#     # rd

#     for _ in range(num_new_per_technique):

#         print('rd'*10)

#         a_words = random_deletion(words, p_rd)

#         print('rd words', a_words)

#         augmented_sentences.append(" ".join(a_words))



#     augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]

#     print('before shuffle', augmented_sentences)

#     random.shuffle(augmented_sentences)

#     print('after shuffle', augmented_sentences)





#     if num_aug >= 1:

#         augmented_sentences = augmented_sentences[:num_aug]

#     else:

#         keep_prob = num_aug / len(augmented_sentences)

#         augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]



#     augmented_sentences.append(sentence)

#     print('증강 완료!!!!!!!!!!!!!!!!!!!!!!!!', augmented_sentences)

#     return augmented_sentences



import random

import pickle

import re



wordnet = {}

with open("/kaggle/input/wordnet/wordnet.pickle", "rb") as f:

	wordnet = pickle.load(f)





# 한글만 남기고 나머지는 삭제

def get_only_hangul(line):

	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)



	return parseText







########################################################################

# Synonym replacement

# Replace n words in the sentence with synonyms from wordnet

########################################################################

def synonym_replacement(words, n):

	new_words = words.copy()

	random_word_list = list(set([word for word in words]))

	random.shuffle(random_word_list)

	num_replaced = 0

	for random_word in random_word_list:

		synonyms = get_synonyms(random_word)

		if len(synonyms) >= 1:

			synonym = random.choice(list(synonyms))

			new_words = [synonym if word == random_word else word for word in new_words]

			num_replaced += 1

		if num_replaced >= n:

			break



	if len(new_words) != 0:

		sentence = ' '.join(new_words)

		new_words = sentence.split(" ")



	else:

		new_words = ""



	return new_words





def get_synonyms(word):

	synomyms = []



	try:

		for syn in wordnet[word]:

			for s in syn:

				synomyms.append(s)

	except:

		pass



	return synomyms



########################################################################

# Random deletion

# Randomly delete words from the sentence with probability p

########################################################################

def random_deletion(words, p):

	if len(words) == 1:

		return words



	new_words = []

	for word in words:

		r = random.uniform(0, 1)

		if r > p:

			new_words.append(word)



	if len(new_words) == 0:

		rand_int = random.randint(0, len(words)-1)

		return [words[rand_int]]



	return new_words



########################################################################

# Random swap

# Randomly swap two words in the sentence n times

########################################################################

def random_swap(words, n):

	new_words = words.copy()

	for _ in range(n):

		new_words = swap_word(new_words)



	return new_words



def swap_word(new_words):

	random_idx_1 = random.randint(0, len(new_words)-1)

	random_idx_2 = random_idx_1

	counter = 0



	while random_idx_2 == random_idx_1:

		random_idx_2 = random.randint(0, len(new_words)-1)

		counter += 1

		if counter > 3:

			return new_words



	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]

	return new_words



########################################################################

# Random insertion

# Randomly insert n words into the sentence

########################################################################

def random_insertion(words, n):

	new_words = words.copy()

	for _ in range(n):

		add_word(new_words)



	return new_words





def add_word(new_words):

	synonyms = []

	counter = 0

	while len(synonyms) < 1:

		if len(new_words) >= 1:

			random_word = new_words[random.randint(0, len(new_words)-1)]

			synonyms = get_synonyms(random_word)

			counter += 1

		else:

			random_word = ""



		if counter >= 10:

			return



	random_synonym = synonyms[0]

	random_idx = random.randint(0, len(new_words)-1)

	new_words.insert(random_idx, random_synonym)







def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):

	sentence = get_only_hangul(sentence)

	words = sentence.split(' ')

	words = [word for word in words if word != ""]

	num_words = len(words)



	augmented_sentences = []

	num_new_per_technique = int(num_aug/4) + 1



	n_sr = max(1, int(alpha_sr*num_words))

	n_ri = max(1, int(alpha_ri*num_words))

	n_rs = max(1, int(alpha_rs*num_words))



	# sr

	for _ in range(num_new_per_technique):

		a_words = synonym_replacement(words, n_sr)

		augmented_sentences.append(' '.join(a_words))



	# ri

	for _ in range(num_new_per_technique):

		a_words = random_insertion(words, n_ri)

		augmented_sentences.append(' '.join(a_words))



	# rs

	for _ in range(num_new_per_technique):

		a_words = random_swap(words, n_rs)

		augmented_sentences.append(" ".join(a_words))



	# rd

	for _ in range(num_new_per_technique):

		a_words = random_deletion(words, p_rd)

		augmented_sentences.append(" ".join(a_words))



	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]

	random.shuffle(augmented_sentences)



	if num_aug >= 1:

		augmented_sentences = augmented_sentences[:num_aug]

	else:

		keep_prob = num_aug / len(augmented_sentences)

		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]



	augmented_sentences.append(sentence)



	return augmented_sentences

아래 코드는 reproducibility를 위한 설정들이고 데이콘에서 제공한 베이스라인 코드에서 따왔습니다.



자세한 설명은 : https://tempdev.tistory.com/28



m1 reproducibility 추가



CFG안에 key, value에 대해서 설명하자면

1. epochs

    - 총 트레이닝을 몇번 반복할지

    - training set를 처음부터 끝까지 도는걸 몇번할지

2. learning_rate

    - 트레이닝 속도

    - 높으면 optimize 값으로 빨리 접근할 수는 있지만 지나칠 수 있다.

    - 낮으면 optimizer 값으로 느리게 접근하지만 지나치지는 않는다. --> training에 소요되는 시간 증가

3. batch_size

    - 한번에 몇개의 training items를 가지고 neural network의 weight를 조정할 것인가

    - 예를 들어 1000개의 items를 training할 때 batch_size가 32라고 하면 1000개를 32개씩 쪼개서 한번에 32개의 item만을 가지고 training하고 weights를 업데이트하고 다음 32개로 넘어간다. 이때 마지막 그룹은 32개보다 적다.

    - batch_size가 작으면 weights를 자주 업데이트하고 크면 weights를 덜 자주 업데이트 한다 --> batch_size가 작으면 sample size가 작은거랑 비슷하므로 그룹마다 차이가 크다 / batch_size가 크면 sample size가 큰거랑 비슷하므로 그룹마다 차이가 작다.





이 코드를 돌린 디바이스가 m1 mac이므로 torch.device('mps')를 썼지만 맥이 아닌 gpu를 쓸때는 'cuda'를 쓰면 된다. gpu가 없다면 'cpu'로 설정하면 되는데 그러면 속도가 많이 느려질 것이다. gpu가 있는지 알아보기 위해서는 torch.cuda.is_available()를 돌려보면 true 아니면 false가 나올 것이다.

In [5]:
def seed_everything(seed):

    random.seed(seed)

    os.environ['PYTHONHASHSEED'] = str(seed)

    np.random.seed(seed)

    torch.manual_seed(seed)

    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.deterministic = True

    torch.backends.cudnn.benchmark = True



CFG = {

    'EPOCHS':20,

    'LEARNING_RATE':1e-5,

    'BATCH_SIZE':32,

    'SEED':41

}



seed_everything(CFG['SEED']) # Seed 고정

device = torch.device('cuda')



# 데이터 증강

# def export_txt(tokenizer, row, aug, max_length) :

#     text = row[0]

#     print('text 확인', text)

#     if not isinstance(text, str):

#         raise ValueError(f"text는 문자열이어야 합니다. 현재 타입: {type(text)}")



#     if (aug > 0) and (aug <=1) :

#         print('진입 text len', len(text))

#         text = EDA(text, alpha_sr=aug, alpha_ri=aug, alpha_rs=aug, p_rd=aug, num_aug=1)

#         print(f"after text: {text}, type: {type(text)}, len(text): {len(text)}")



#     else :

#         pass

#     inputs = tokenizer(

#         text,

#         return_tensors='pt',

#         truncation=True,

#         max_length=max_length,

#         pad_to_max_length=True,

#         add_special_tokens=True

#         )

#     print('after tokenizer', inputs)

#     input_ids = inputs['input_ids'][0]

#     attention_mask = inputs['attention_mask'][0]

#     return input_ids, attention_mask

def augment_data(df, text_column, label_column, num_aug=5):

    augmented_data = []

    for _, row in df.iterrows():

        augmented_sentences = EDA(row[text_column], num_aug=num_aug)

        # print('증강',augmented_sentences)

        for sentence in augmented_sentences:

            # print(sentence)

            new_row = {text_column: sentence}  # Initialize new_row with the text_column and sentence

            for label in label_column:  # Iterate through label_columns

                new_row[label] = row[label]  # Add each label column to new_row

            # new_row = {text_column: sentence, label_column: row[label_column]}



            # new_row = row[label_column].to_dict()  # Convert selected columns to a dictionary

            # new_row[text_column] = sentence        # Add the augmented sentence

            # print(new_row)

            augmented_data.append(new_row)         # Append the new row to the list

    return pd.DataFrame(augmented_data)

그 다음으로는 train_test_split를 이용해서 train와 val을 나눠줍니다.



유형(type), 극성(polarity), 시제(tense), 확실성(certainty) 중에서 imbalanced한 label도 있기때문에



stratify를 추가해주는게 좋다.

In [6]:
text_column = '문장'

label_column = ['유형','극성', '시제', '확실성','label']

augmented_data = augment_data(train_original, text_column, label_column)

augmented_data.shape

(99246, 6)

In [7]:
augmented_data.head()

,문장,유형,극성,시제,확실성,label
0,0.75%포인트 금리 인상은 1994년 28년 이후 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
2,인상은 1994년 이후 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
3,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
4,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [8]:
train, val, train_target, val_target = train_test_split(augmented_data.drop('label', axis=1), augmented_data['label'], test_size=0.2, random_state=CFG['SEED'])

train = train.reset_index(drop=True)

val = val.reset_index(drop=True)

train_target = train_target.reset_index(drop=True)

val_target = val_target.reset_index(drop=True)

train.shape, val.shape, train_target.shape, val_target.shape

((79396, 5), (19850, 5), (79396,), (19850,))

In [9]:
train

,문장,유형,극성,시제,확실성
0,당시 청소년의 위해 보장하기 수면권을 대통령령이 정하는 심야시간에 발의된 대한 게임...,사실형,긍정,과거,확실
1,낙화생은 옹크려져 누에 같으면서 몸뚱이가 형체가 있다.,사실형,긍정,현재,확실
2,휴일이면 자동차를 자가 정비하는 모습을 동네 곳곳에서 쉽게 볼 수 있다.,추론형,긍정,현재,확실
3,그러나 백병전 답 익숙한 지형에 매복해 활을 쏘는 게릴라전에서는 승전 소식이 들리기...,사실형,긍정,과거,확실
4,횡령 추정액 중 2017∼2019년 사이 사라진 3억4000만원은 지난달 확인됐다....,사실형,긍정,과거,확실
...,...,...,...,...,...
79391,한국 정부도 노인 일자리를 만들기 위 위해 노력해왔다.,사실형,긍정,과거,확실
79392,그런데 이렇게 맛 맛 좋은 우곡주를 제가 잘 팔지 못했어요.,대화형,부정,과거,확실
79393,셀트리온이 밝힌 공식적인 블록딜 사유는 ＇투자 기간별 포트폴리오 조정＇이었다.,사실형,긍정,과거,확실
79394,화장실은 단지 배 속 평안을 간구하는 생리학적 배 공간이 아니라 인류의 평안을 추구...,추론형,긍정,현재,확실


위에서 언급했듯이 transformers 패키지를 통해서 pretrained된 모델을 불러올 수 있다.



불러오는 방법은 https://huggingface.co/models 여기서 model 이름을 검색하고



모델은 AutoModel.from_pretrained()을 통해서, 토크나이저는 AutoTokenizer.from_pretrained()을 통해서



모델이름을 파라미터로 넣어주면 된다.

In [10]:
model_nm = 'klue/roberta-small'

base_model = AutoModel.from_pretrained(model_nm)

tokenizer = AutoTokenizer.from_pretrained(model_nm)

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

토크나이저의 역할은 문장을 토큰이라고 하는 작은 단위 (더이상 나눌 수 없는 가장 작은 단위)로 나누어주고 pretrained tokenizer에 그 토큰이 어디에 저장되어 있는지 input_ids로 되돌려준다.



밑 코드는 그 input_ids 길이가 어떤지 histogram으로 그려본 것이다.



(저도 이 분야는 신생아라 틀린 부분이 있을 수 있습니다.)

In [11]:
# tokenizer_len = [len(tokenizer(s)['input_ids']) for s in train['문장']]

# sns.histplot(tokenizer_len)

# plt.show()



# print(f'log value : {np.mean(tokenizer_len)+3*np.std(tokenizer_len)}')

따라서, log를 취해주면

In [12]:
# tokenizer_log = np.log(tokenizer_len)

# sns.histplot(tokenizer_log)

# plt.show()



# print(f'log value : {np.mean(tokenizer_log)+3*np.std(tokenizer_log)}')

# print(f'original value : {np.exp(np.mean(tokenizer_log)+3*np.std(tokenizer_log))}')

따라서, 적정선은 90에서 134 사이가 아닐까 생각된다.

위 값은 잠깐 뒤로하고 아래 코드를 위부터 설명하자면,



torch.utils.data의 Dataset의 child class인 SentenceTypeDataset을 만들어준다. 이는 pytorch neural network에 내 맘대로

x가 뭔지 y가 뭔지 정한 dataset을 제공하기 위한 클래스이다. 만드는데 꼭 필요한 function은 총 3가지이다.



_ _ init _ _()

- x랑 y가 뭔지 저장해줘야 한다.

- 이때 이번에는 텍스트 데이터를 다뤄주기때문에 dataframe, tokenizer, labels를 parameter로 넣어줬고 dataframe의 문장을 tokenizer로 토큰화 시킨다음 self.texts에 저장해주고 입력받은 labels는 그대로 self.labels에 저장해줬다.

- batch에 들어가는 입력들은 input size가 다 같아야 한다. 따라서 tokenizer로 나온 값들을 그대로 넣어버리면 오류가 난다. 위 히스토그램에서 볼 수 있듯이 문장마다 다르기 때문. 따라서 위 값(90)으로 tokenizer의 max_length를 정해주고 max_length보다 작은 길이들은 padding으로 채워주고 긴 길이들은 truncation으로 잘라준다. 이때 max_length는 길면 길수록 training time이 늘어난다.

- tokenizer안에 return_tensors의 pt는 tensor로 tokenizer값을 리턴해준다는 뜻이다.



_ _ len _ _()

- self.texts의 길이는 리턴해줍니다.



_ _ getitem _ _()

- idx에 해당되는 x와 y를 리턴해줍니다.

- x는 self.texts에서 가져오고 y는 그 텍스트(x)에 해당되는 type, polarity, tense, certainty를 리턴하면 됩니다.

- 나중 코드에서 나오겠지만 미리 설명을 하자면 labels는 train, val set에서만 dictionary형태로 주어진다. 이때 type, polarity, tense, certainty가 key고 해당 레이블에 해당되는 값을 one-hot encoding한게 value다.

- dictionary에 저장된 형태는 list고 pytorch에는 tensor 형태로 넣어줘야하기 때문에 torch.Tensor()로 형태를 바꾸어준다.

- test set의 경우 labels이 주어지지 않기 때문에 똑같은 길이지만 -1로 채워놓은 tensor를 리턴해준다.

In [13]:
class SentenceTypeDataset(Dataset):

    def __init__(self, dataframe, tokenizer, labels=None):

        texts = dataframe['문장'].values.tolist()

        # print('texts', texts)

        self.texts = [tokenizer(text, padding='max_length', max_length=90, truncation=True, return_tensors='pt') for text in texts]

        self.labels = labels



    def __len__(self):

        return len(self.texts)



    def __getitem__(self, idx):

        text = self.texts[idx]

        # print('text********************\n', text)

        if self.labels is not None:

            type_tmp = self.labels['type'][idx]

            polarity_tmp = self.labels['polarity'][idx]

            tense_tmp = self.labels['tense'][idx]

            certainty_tmp = self.labels['certainty'][idx]

            return text, torch.Tensor(type_tmp), torch.Tensor(polarity_tmp), torch.Tensor(tense_tmp), torch.Tensor(certainty_tmp)

        else:

            return text, torch.Tensor([-1,-1,-1,-1]), torch.Tensor([-1,-1,-1]), torch.Tensor([-1,-1,-1]), torch.Tensor([-1,-1])



# class SentenceTypeDataset(Dataset):

#     def __init__(self, df, tokenizer, labels, max_length = 512, aug = 1):

#         self.max_length = max_length

#         self.dataset = df.dropna(axis=0)

#         self.dataset['label_0'] =  pd.DataFrame(labels).iloc[:,0]

#         self.dataset['label_1'] =  pd.DataFrame(labels).iloc[:,1]

#         self.dataset['label_2'] =  pd.DataFrame(labels).iloc[:,2]

#         self.dataset['label_3'] =  pd.DataFrame(labels).iloc[:,3]

#         self.dataset = self.dataset.rename(columns = {'문장' : 'document'})

#         # self.dataset['ID'] = self.dataset['ID'].apply(make_id)

#         self.dataset = self.dataset[['document', 'label_0', 'label_1', 'label_2', 'label_3']]

#         self.tokenizer = tokenizer

#         self.aug = aug

#         print('found {} data'.format(len(self.dataset)))

#         print(self.dataset.describe())



#     def __len__(self):

#         return len(self.dataset)



#     def __getitem__(self, idx):

#         # row에 문장도 포함해야함... 근데 없어...

#         row = self.dataset.iloc[idx, :].values

#         print('row', row)

#         print(row[1], row[2], row[3], row[4])

#         input_ids, attention_mask = export_txt(self.tokenizer, row, self.aug, self.max_length)

#         print('input_ids', input_ids)

#         print('attention_mask', attention_mask)

#         y0, y1, y2, y3 = row[1], row[2], row[3], row[4]

#         return {

#             'input_ids':  input_ids,

#             'attention_mask': attention_mask,

#             'y0':y0, 'y1':y1, 'y2':y2, 'y3':y3

#         }



# class SentenceTypeDataset(Dataset):

#     def __init__(self, texts, tokenizer, labels, max_len):

#         self.texts = texts['문장'].tolist()  # Access the '문장' column and convert it to a list

#         self.labels = labels

#         self.tokenizer = tokenizer

#         self.max_len = max_len



#     def __len__(self):

#         return len(self.texts)



#     def __getitem__(self, idx):

#         text = self.texts[idx]

#         # Get labels based on 'idx' instead of accessing label dict directly.

#         type_label = self.labels['type'][idx]

#         polarity_label = self.labels['polarity'][idx]

#         tense_label = self.labels['tense'][idx]

#         certainty_label = self.labels['certainty'][idx]

#         assert len(self.labels['type']) == len(self.labels['polarity']) == len(self.labels['tense']) == len(self.labels['certainty']), "라벨 길이가 다릅니다."



#         encoding = self.tokenizer(

#             text,

#             max_length=self.max_len,

#             padding="max_length",

#             truncation=True,

#             return_tensors="pt",

#         )

#         return {

#             "input_ids": encoding["input_ids"].squeeze(0),

#             "attention_mask": encoding["attention_mask"].squeeze(0),

#             "label": torch.tensor([type_label, polarity_label, tense_label, certainty_label], dtype=torch.long)

#             }


다음으로는 Classifier class를 만드는 것이다.



\__init__()

- 우선 nn.Module의 child class이기 때문에 \__init__() 안에 super().\__init__()을 불러주고 base_model을 통해서 받을 pretrained_model을 self.klue에 저장한다.

이때 klue의 output features는 768이다. 확인방법은 base_model을 셀에 쳐보면 된다. (더 쉬운 방법이 있으면 알려주세요)



- transfer learning의 기본적인 방법은 중간에 hidden layer는 pretrained_model의 것을 이용하고 output layer를 내가 원하는 방향으로 만들어서 training 하는 것이다. 따라서, self.fc1, self.type_clf, self.softmax 등등 다양한 레이어들을 추가해줬다. 이때 self.fc1에는 nn.Linear(768, 32)를 저장해줬는데, 이는 in_feature로 768, out_feature로는 32를 내보낸다는 뜻이다. 일반적으로 알고 있는 dense layer의 역할을 한다. 그 다음으로는 self.relu에 nn.ReLU()를 저장해서 activation function으로 사용해줬다. 그 다음으로는 multilabel classification 문제이기 때문에 각 label마다 nn.Linear(32, # of types)으로 레이어를 만들어줬다. 이때 # of types만큼의 out_feature가 필요한 이유는 types들을 one-hot encoding을 해줬기 때문이다. 그 다음으로는 classification에 많이 사용되는 nn.Softmax(dim=1)을 넣어줬다. softmax에서 나온 값들의 합은 1로써 어느 type에 해당되는지 확률들을 리턴해준다. 이때 합해져야되는 값들이 dim=1에 있기때문에 dim=1이라는 파라미터를 넣어주었다.



\__forward__()

- 그 다음으로 꼭 작성해줘야 하는 function은 forward다. (backward는 필요없음) 여기서는 위에서 작성한 레이어들을 어느 순서로 지나칠지 순서를 정해주는 단계이다. 우선, pretrained_model을 지나고 나온 output을 fc1과 relu에 넘겨주고 그 다음으로는 각 label의 clf-softmax 페어를 지나쳐준다. 그리고 나온 4개의 output을 리턴해주면 된다.

- multilabel이기 때문에 4개의 값을 리턴해주는거지 단순하게 binary 또는 multiclassification이면 보통 1개의 output만을 리턴해주면 된다.

- 아래 코드에 써있듯이 input_ids는 토큰에 해당되는 ids들, attention_mask는 어느 토큰에 집중해야되는지 알려주는 역할을 한다. 이는 왜 필요하나 하면 padding 단계에서 추가된 padding token에 대한 접근을 막기위해 사용된다.

In [14]:
class SentenceClassifier(nn.Module):

    def __init__(self, base_model):

        super().__init__()

        self.klue = base_model # from transformers package



        self.fc1 = nn.Linear(self.klue.config.hidden_size, 32)

        self.layernorm = nn.LayerNorm(32)

        self.dropout = nn.Dropout(p = 0.1)

        self.relu = nn.ReLU()

        self.type_clf = nn.Linear(32,4)

        self.polarity_clf = nn.Linear(32,3)

        self.tense_clf = nn.Linear(32,3)

        self.certainty_clf = nn.Linear(32,2)

        self.softmax = nn.Softmax(dim=1)



    def forward(self, input_ids, attention_mask):

        # input_ids : token's id / attention_mask : make a model to focus on which token

        klue_out = self.klue(input_ids= input_ids, attention_mask = attention_mask)[0][:,0]



        x = self.fc1(klue_out)

        # print('x*************************', x)

        x = self.layernorm(x)

        x = self.dropout(x)

        x = self.relu(x)



        type_output = self.type_clf(x)

        type_output = self.softmax(type_output)

        polarity_output = self.polarity_clf(x)

        polarity_output = self.softmax(polarity_output)

        tense_output = self.tense_clf(x)

        tense_output = self.softmax(tense_output)

        certainty_output = self.certainty_clf(x)

        certainty_output = self.softmax(certainty_output)



        return type_output, polarity_output, tense_output, certainty_output

다음으로는! training 단계이다.



우선 val_loss를 기준으로 early_stop을 할건지 말건지 정하기 때문에 best_val_loss를 설정해주었고, crossentropyloss를 이용할건데 작아질수록 좋은 값이기 때문에 최초값은 높은 값으로 설정해주었다.



그 다음으로는 criterion인데 이는 loss function이다. 4개의 다른 label들이 있기 때문에 dictionary에 4개를 넣어주었다. 나중 단계에서 criterion에 있는 CrossEntropyLoss를 통해서 true값과 pred값의 차이를 구하고 어떤 방향으로 weights를 조정해야되는지 정한다.



optimizer는 어떤 방식으로 최적화를 한걸지 정해주는 변수인데, 일반적으로 많이 쓰이는 Adam을 써줬다. Adam 안에는 모델의 파라미터(model.parameters())와 learning_rate를 넣어주었다. 위에서 언급했듯이 이때 learning_rate가 큰지 작은지에 따라 training 속도가 결정난다. 그 다음으로는 모델을 gpu로 보내주었다. 이 코드가 있어야 gpu를 사용해서 training 한다.

- mac m칩 유저의 경우 “PYTORCH_ENABLE_MPS_FALLBACK=1”를 설정해줘야 mps가 안되는 코드는 cpu로 계산을 해준다. (2022/12/16 cumsum은 mps로 계산이 안됨)



그리고 주어진 epochs만큰 for loop을 돌리는데 그 밑에 있는 total_acc_train은 total_f1_train으로 바뀌어야 맞다. 이 부분은 중간에 f1 계산하는 코드 넣는거를 까먹고 못하고 코드를 그대로 돌려서 남은 것이니 만약 이 코드 그대로 돌린다고 하면 바꾸어주길 바란다. 그 밑에 total_loss_train은 epoch별로 loss 값이 어땠는지 기록해주기 위해 만든 변수다.



그 다음으로는 model.train()이 있는데 이는 model을 training 모드로 만들어주는거다. 이렇게 해야 weight들이 업데이트된다. 이와 반대로 나중에 val이나 test set을 모델에 넘겨줄때는 model.eval()을 불러줘야한다. 이래야 weights들이 업데이트 되지 않는다.



그 다음으로는 train_dataloader를 for loop으로 돌려주는데 뒤에 나오겠지만 dataloader는 지정해준 batch_size만큼 item의 x와 y를 넘겨준다. 이때 쓰이는 것이 위에서 만들어준 SentenceTypeDataset의 getitem()이다. 따라서, 5개의 변수 (train_input, type_label, polarity_label, tense_label, certainty_label)로 받아야한다. 그 다음으로는 train_input에 있는 attention_mask와 input_ids와 label들을 device로 넘겨준다.



그리고 training을 시작하기 전에 optimizer.grad()를 설정해줘서 매 epoch마다 전에 썼던 값들을 기억하는 것이 아니라 0 베이스에서 시작하게 해준다. epoch를 통한 값들의 정확한 업데이트를 위해서는 꼭 필요한 코드다.



그리고 나서 model에 input_ids와 attention_mask를 넣어서 얻은 4개의 값들을 저장해준다. 이때 이 값들은 각 label마다 one-hot encoding된 컬럼에 해당될 확률들이다. 바로 다음에 이 값들은 criterion에 있는 CrossEntropyLoss()로 들어가서 실제와 얼마나 유사한지 계산되고 그 계산된 값을 total_loss_train에 저장해준다.



그 다음으로는 계산된 loss 값을 바탕으로 backpropagation(loss.backward()과 optimizer.step()을 통해서)을 진행하여 weights들을 업데이트해준다.



이렇게 training data를 다 거쳤다면 그 다음은 validation data 차례다. 우선 with torch.no_grad()과 model.eval()을 불러주어서 weights들을 업데이트하는게 아니라는 것을 선언해준다. 그 후에는 training data에서 했던 방식이랑 다 같지만 optimizer.zero_grad(), loss.backward(), optimier.step()만 빠진다. weights들을 업데이트하지 않기 때문.



그 다음으로는 지금까지 저장한 loss와 metric을 프린트해주고, val_loss가 좋아졌는지 여부에 따라서 모델을 저장할 것인지 early stop 할 것인지 정해준다.

In [18]:
from sklearn.metrics import f1_score

import numpy as np

import torch



def sentence_train(model, train_dataloader, val_dataloader, learning_rate, epochs, model_nm):

    best_val_loss = 99999999999999  # setting max (act as infinity)

    early_stopping_threshold_count = 0



    criterion = {

        'type': nn.CrossEntropyLoss().to(device),

        'polarity': nn.CrossEntropyLoss().to(device),

        'tense': nn.CrossEntropyLoss().to(device),

        'certainty': nn.CrossEntropyLoss().to(device)

    }



    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model = model.to(device)



    for epoch in range(epochs):

        total_f1_train = 0  # F1 score for training set

        total_loss_train = 0



        model.train()  # sets into the training mode



        all_true_labels = []

        all_pred_labels = []



        for train_input, type_label, polarity_label, tense_label, certainty_label in tqdm(train_dataloader):

            attention_mask = train_input['attention_mask'].to(device)

            input_ids = train_input['input_ids'].squeeze(1).to(device)

            type_label = type_label.to(device)

            polarity_label = polarity_label.to(device)

            tense_label = tense_label.to(device)

            certainty_label = certainty_label.to(device)

            optimizer.zero_grad()  # 가중치를 업데이트할 때 사용할 기울기를 초기화



            type_output, polarity_output, tense_output, certainty_output = model(input_ids, attention_mask)  # from the forward function

            # print('output***********************************************', type_output, polarity_output, tense_output, certainty_output)

            loss = 0.25 * criterion['type'](type_output, type_label) + 0.25 * criterion['polarity'](polarity_output, polarity_label) + 0.25 * criterion['tense'](tense_output, tense_label) + 0.25 * criterion['certainty'](certainty_output, certainty_label)

            total_loss_train += loss.item()



            loss.backward()

            optimizer.step()



            # Collect true labels and predicted labels for F1 score calculation

            all_true_labels.append(type_label.cpu().numpy())  # True labels

            all_pred_labels.append(torch.argmax(type_output, dim=1).cpu().numpy())  # Predicted labels



        # Flatten the list of true and predicted labels



        all_true_labels = np.argmax(np.concatenate(all_true_labels), axis=1) if all_true_labels[0].ndim > 1 else np.concatenate(all_true_labels)

        all_pred_labels = np.argmax(np.concatenate(all_pred_labels), axis=1) if all_pred_labels[0].ndim > 1 else np.concatenate(all_pred_labels)



        print(all_true_labels)

        print(all_pred_labels)

        # Compute F1 score for training

        total_f1_train = f1_score(all_true_labels, all_pred_labels, average='weighted')  # 'weighted' for multiclass



        with torch.no_grad():  # since we should not change gradient for validation

            total_f1_val = 0  # F1 score for validation set

            total_loss_val = 0



            model.eval()  # deactivate training



            all_true_val_labels = []

            all_pred_val_labels = []



            for val_input, vtype_label, vpolarity_label, vtense_label, vcertainty_label in tqdm(val_dataloader):

                attention_mask = val_input['attention_mask'].to(device)

                input_ids = val_input['input_ids'].squeeze(1).to(device)



                vtype_label = vtype_label.to(device)

                vpolarity_label = vpolarity_label.to(device)

                vtense_label = vtense_label.to(device)

                vcertainty_label = vcertainty_label.to(device)



                vtype_output, vpolarity_output, vtense_output, vcertainty_output = model(input_ids, attention_mask)  # from the forward function



                loss = 0.25 * criterion['type'](vtype_output, vtype_label) + 0.25 * criterion['polarity'](vpolarity_output, vpolarity_label) + 0.25 * criterion['tense'](vtense_output, vtense_label) + 0.25 * criterion['certainty'](vcertainty_output, vcertainty_label)



                total_loss_val += loss.item()



                # Collect true labels and predicted labels for F1 score calculation

                all_true_val_labels.append(vtype_label.cpu().numpy())  # True labels

                all_pred_val_labels.append(torch.argmax(vtype_output, dim=1).cpu().numpy())  # Predicted labels



            # Flatten the list of true and predicted labels

            all_true_val_labels = np.argmax(np.concatenate(all_true_val_labels), axis=1) if all_true_val_labels[0].ndim > 1 else np.concatenate(all_true_val_labels)

            all_pred_val_labels = np.argmax(np.concatenate(all_pred_val_labels), axis=1) if all_pred_val_labels[0].ndim > 1 else np.concatenate(all_pred_val_labels)



            print(all_true_val_labels)

            print(all_pred_val_labels)

            # Compute F1 score for validation

            total_f1_val = f1_score(all_true_val_labels, all_pred_val_labels, average='weighted')  # 'weighted' for multiclass



            os.makedirs("model", exist_ok=True)



            print(f'Epochs: {epoch + 1} '

                  f'| Train Loss: {total_loss_train / len(train_dataloader): .3f} '

                  f'| Train F1 Score: {total_f1_train: .3f} '

                  f'| Val Loss: {total_loss_val / len(val_dataloader): .3f} '

                  f'| Val F1 Score: {total_f1_val: .3f}')



            if best_val_loss > total_loss_val:

                best_val_loss = total_loss_val  # saving only the best one

                torch.save(model, f"model/{model_nm}.pt")

                print("Saved model")

                early_stopping_threshold_count = 0

            else:

                early_stopping_threshold_count += 1  # checking how many epochs have passed that val_loss didn't increase



            if early_stopping_threshold_count >= 3:  # ==> patience=1

                print("Early stopping")

                break


필요한 class, function들은 다 적었으니 이제 dataset들을 준비할 차례다.



우선 train에서 label을 제외한 나머지 컬럼들만 킵해주고 그중에서도 유형,극성,시제,확실성을 pd.get_dummies로 one-hot encoding해준다.

In [19]:
print('before shape', train.shape)

train_tmp = train[['문장', '유형', '극성', '시제', '확실성']]

train_tmp = pd.get_dummies(train_tmp, columns=['유형', '극성', '시제', '확실성'])

print('train_tmp', train_tmp.head())



print('after shape', train_tmp.shape)

before shape (79396, 5)
train_tmp                                                   문장  유형_대화형  유형_사실형  유형_예측형  \
0  당시 청소년의 위해 보장하기 수면권을 대통령령이 정하는 심야시간에 발의된 대한 게임...   False    True   False   
1                     낙화생은 옹크려져 누에 같으면서 몸뚱이가 형체가 있다.   False    True   False   
2           휴일이면 자동차를 자가 정비하는 모습을 동네 곳곳에서 쉽게 볼 수 있다.   False   False   False   
3  그러나 백병전 답 익숙한 지형에 매복해 활을 쏘는 게릴라전에서는 승전 소식이 들리기...   False    True   False   
4  횡령 추정액 중 2017∼2019년 사이 사라진 3억4000만원은 지난달 확인됐다....   False    True   False   

   유형_추론형  극성_긍정  극성_미정  극성_부정  시제_과거  시제_미래  시제_현재  확실성_불확실  확실성_확실  
0   False   True  False  False   True  False  False    False    True  
1   False   True  False  False  False  False   True    False    True  
2    True   True  False  False  False  False   True    False    True  
3   False   True  False  False   True  False  False    False    True  
4   False   True  False  False   True  False  False    False    True  
after shape (79396, 13)


그 다음으로는 각 label별로 뽑아서 train_labels에 dictionary형태로 저장해준다.

In [20]:
train_type = train_tmp.iloc[:,1:5].values.tolist()

train_polarity = train_tmp.iloc[:,5:8].values.tolist()

train_tense = train_tmp.iloc[:,8:11].values.tolist()

train_certainty = train_tmp.iloc[:,11:13].values.tolist()



train_labels = {

    'type': train_type,

    'polarity': train_polarity,

    'tense': train_tense,

    'certainty': train_certainty

}

pd.DataFrame(train_labels)

,type,polarity,tense,certainty
0,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
1,"[False, True, False, False]","[True, False, False]","[False, False, True]","[False, True]"
2,"[False, False, False, True]","[True, False, False]","[False, False, True]","[False, True]"
3,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
4,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
...,...,...,...,...
79391,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
79392,"[True, False, False, False]","[False, False, True]","[True, False, False]","[False, True]"
79393,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
79394,"[False, False, False, True]","[True, False, False]","[False, False, True]","[False, True]"


똑같은 방식으로 validation data도 만든다.

In [21]:
val_tmp = val[['문장', '유형', '극성', '시제', '확실성']]

val_tmp = pd.get_dummies(val_tmp, columns=['유형', '극성', '시제', '확실성'])

# print('val_tmp', val_tmp.head())

val_type = val_tmp.iloc[:,1:5].values.tolist()

val_polarity = val_tmp.iloc[:,5:8].values.tolist()

val_tense = val_tmp.iloc[:,8:11].values.tolist()

val_certainty = val_tmp.iloc[:,11:13].values.tolist()

val_labels = {

    'type': val_type,

    'polarity': val_polarity,

    'tense': val_tense,

    'certainty': val_certainty

}

pd.DataFrame(val_labels)

,type,polarity,tense,certainty
0,"[False, True, False, False]","[True, False, False]","[False, False, True]","[False, True]"
1,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
2,"[False, True, False, False]","[True, False, False]","[False, False, True]","[False, True]"
3,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
4,"[False, True, False, False]","[True, False, False]","[False, True, False]","[False, True]"
...,...,...,...,...
19845,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"
19846,"[False, True, False, False]","[True, False, False]","[False, False, True]","[False, True]"
19847,"[False, True, False, False]","[True, False, False]","[False, False, True]","[False, True]"
19848,"[False, True, False, False]","[True, False, False]","[True, False, False]","[False, True]"


train, val set가 준비되었다면 위에서 만든 SetenceTypeDataset에 dataframe, tokenizer, labels들을 넣어주고 SentenceTypeDataset, batch_size는 필수적으로 값을 정해서 DataLoader에 넣어준다. 위에서 잠깐 언급했듯이 DataLoader는 지정된 batch_size만큼 item들을 모델에 넘겨주어서 training 할 수 있게 해준다.



여기서 shuffle는 item들을 랜덤하게 고른다는 의미고 num_workers는 설명을 봐도 잘 모르겠다.

In [22]:
pd.DataFrame(train_labels).iloc[:, 1]

0        [True, False, False]
1        [True, False, False]
2        [True, False, False]
3        [True, False, False]
4        [True, False, False]
                 ...         
79391    [True, False, False]
79392    [False, False, True]
79393    [True, False, False]
79394    [True, False, False]
79395    [True, False, False]
Name: polarity, Length: 79396, dtype: object

In [23]:
train_dataloader = DataLoader(SentenceTypeDataset(train_tmp, tokenizer, train_labels), batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0) # num_workers: how many subprocesses to use for data loading

val_dataloader = DataLoader(SentenceTypeDataset(val_tmp, tokenizer, val_labels), batch_size=CFG['BATCH_SIZE'], num_workers=0)

그 다음으로는 base_model (klue를 이용한 pretrained_model)을 기반으로해서 SentenceClassifier를 불러준다.

In [24]:
model = SentenceClassifier(base_model)

그리고 이제서야 training을 할 수 있는 상태에 온 것이다.



위에서 만든 sentence_train function에 필요한 파라미터들을 보내주어서 training을 시작해준다.



(위에서 말했듯이 f1을 계산 안하고 accuracy를 그대로 남겨주었기 때문에 accuracy는 계속 0이다.)

In [25]:
print(len(train_dataloader.dataset))  # 데이터셋 크기 확인

print(len(val_dataloader.dataset))    # 데이터셋 크기 확인

79396
19850


In [26]:
for batch in train_dataloader:

    print('batch', batch)  # 데이터가 올바르게 로드되는지 확인

    break

batch [{'input_ids': tensor([[[    0,  3625,    23,  ...,     1,     1,     1]],

        [[    0,     3, 26947,  ...,     1,     1,     1]],

        [[    0,  6910,  2259,  ...,     1,     1,     1]],

        ...,

        [[    0, 11821,  6724,  ...,     1,     1,     1]],

        [[    0, 25668,  2850,  ...,     1,     1,     1]],

        [[    0,   170,  1583,  ...,     1,     1,     1]]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]]), 'attention_mask': tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]])}, tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [

In [28]:
sentence_train(model, train_dataloader, val_dataloader, CFG['LEARNING_RATE'], CFG['EPOCHS'], 'kclue')

100%|██████████| 2482/2482 [09:45<00:00,  4.24it/s]


[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


100%|██████████| 621/621 [00:47<00:00, 13.19it/s]


[1 1 1 ... 1 1 2]
[1 1 1 ... 1 1 3]
Epochs: 1 | Train Loss:  0.822 | Train F1 Score:  0.786 | Val Loss:  0.772 | Val F1 Score:  0.823
Saved model


100%|██████████| 2482/2482 [09:50<00:00,  4.21it/s]


...

[1 1 1 ... 1 1 2]
[1 1 1 ... 1 1 2]
Epochs: 19 | Train Loss:  0.559 | Train F1 Score:  0.978 | Val Loss:  0.558 | Val F1 Score:  0.977
Saved model


100%|██████████| 2482/2482 [09:49<00:00,  4.21it/s]


[1 3 1 ... 1 1 1]
[1 3 1 ... 1 1 1]


100%|██████████| 621/621 [00:47<00:00, 13.18it/s]

[1 1 1 ... 1 1 2]
[1 1 1 ... 1 1 2]
Epochs: 20 | Train Loss:  0.558 | Train F1 Score:  0.979 | Val Loss:  0.559 | Val F1 Score:  0.977


training이 끝났으면 test data를 이용해 예측을 해야되는데 이때 방법은 validation때와 비슷하다. 따라서, 설명은 생략하겠습니다.

In [31]:
def get_type_predictions(model, loader):



    device = torch.device('cuda')

    model = model.to(device)



    type_probs, polarity_probs, tense_probs, clarity_probs = [], [], [], []

    with torch.no_grad():

        model.eval()

        for data_input, _, _, _, _ in tqdm(loader):

            attention_mask = data_input['attention_mask'].to(device)

            input_ids = data_input['input_ids'].squeeze(1).to(device)





            type_output, polarity_output, tense_output, clarity_output = model(input_ids, attention_mask)

            type_probs.append(type_output)

            polarity_probs.append(polarity_output)

            tense_probs.append(tense_output)

            clarity_probs.append(clarity_output)



    return torch.cat(type_probs).cpu().detach().numpy(), torch.cat(polarity_probs).cpu().detach().numpy(), torch.cat(tense_probs).cpu().detach().numpy(), torch.cat(clarity_probs).cpu().detach().numpy()

In [32]:
model = torch.load("model/kclue.pt")

test_dataloader = DataLoader(SentenceTypeDataset(test, tokenizer), batch_size=CFG['BATCH_SIZE'], shuffle=False)

/tmp/ipykernel_23/2629143655.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model/kclue.pt")


In [33]:
#val_pred_type, val_pred_polarity, val_pred_tense, val_pred_certainty = get_type_predictions(model, val_dataloader)



#val_type = ['대화형' if i==0 else '사실형' if i==1 else '예측형' if i==2 else '추론형' for i in [np.argmax(p) for p in val_pred_type]]

#val_polarity = ['긍정' if i==0 else '미정' if i==1 else '부정' for i in [np.argmax(p) for p in val_pred_polarity]]

#val_type = ['과거' if i==0 else '미래' if i==1 else '현재' for i in [np.argmax(p) for p in val_pred_tense]]

#val_type = ['불확실' if i==0 else '확실' for i in [np.argmax(p) for p in val_pred_certainty]]

In [34]:
test_pred_type, test_pred_polarity, test_pred_tense, test_pred_certainty = get_type_predictions(model, test_dataloader)

100%|██████████| 222/222 [00:16<00:00, 13.56it/s]


여기서 잠깐 test_pred_tense가 어떻게 생겼는지 살펴보면

In [35]:
test_pred_tense

array([[8.8705689e-05, 1.2758546e-04, 9.9978381e-01],
       [8.7229186e-05, 1.2457263e-04, 9.9978822e-01],
       [9.9982470e-01, 6.3605949e-05, 1.1173662e-04],
       ...,
       [3.3157936e-03, 9.9397248e-01, 2.7117103e-03],
       [2.0267006e-02, 9.3503118e-01, 4.4701785e-02],
       [9.9982858e-01, 6.2598578e-05, 1.0876919e-04]], dtype=float32)

위에 보이는 것과 같이 tense에는 3개의 타입이 있으므로 3개의 컬럼들을 볼 수 있다. 그리고 각 컬럼에 해당될 확률이 어느정도인지 저장되어 있는 형태이다. 이때 같은 row에 위치한 값들을 더해보면

In [36]:
sum(test_pred_tense[0])

1.0000001051012077

1인것을 알 수 있다. 위에 softmax에서 설명한 그대로이다.



따라서, 이제 이 값들을 np.argmax()를 통해서 어느 인덱스에 최고 값이 있는지 알아보고 그 인덱스에 맞춰서 맞는 label로 변형해줘야 한다.

그 후 저장하면 제출할 수 있는 파일이 만들어진다.

In [37]:
test_type = ['대화형' if i==0 else '사실형' if i==1 else '예측형' if i==2 else '추론형' for i in [np.argmax(p) for p in test_pred_type]]

test_polarity = ['긍정' if i==0 else '미정' if i==1 else '부정' for i in [np.argmax(p) for p in test_pred_polarity]]

test_tense = ['과거' if i==0 else '미래' if i==1 else '현재' for i in [np.argmax(p) for p in test_pred_tense]]

test_certainty = ['불확실' if i==0 else '확실' for i in [np.argmax(p) for p in test_pred_certainty]]

In [38]:
label_sum = []

for i in range(len(test_type)):

    label_sum.append(f'{test_type[i]}-{test_polarity[i]}-{test_tense[i]}-{test_certainty[i]}')

os.makedirs('submission', exist_ok=True)



submission['label'] = label_sum

submission.to_csv('submission/klue1.csv', index=False)

In [39]:
submission

,ID,label
0,TEST_0000,사실형-긍정-현재-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-과거-확실
4,TEST_0004,사실형-긍정-과거-확실
...,...,...
7085,TEST_7085,사실형-긍정-현재-확실
7086,TEST_7086,추론형-긍정-현재-확실
7087,TEST_7087,사실형-긍정-미래-확실
7088,TEST_7088,추론형-긍정-미래-확실


In [ ]:
import joblib

torch.save(model.state_dict(), "klue_state_dict.pth")

joblib.dump("klue_state_dict.pth", './klue_model.pkl')

In [ ]:
import torch

import joblib



# joblib으로 압축된 state_dict 파일을 불러옴

state_dict_path = joblib.load("./klue_model.pkl")

model.load_state_dict(torch.load(state_dict_path))

model.eval()

In [ ]:
for name, param in model.named_parameters():

    print(f"{name}: {param[0]}")

    break  # 첫 번째 파라미터만 출력

궁금한 부분이나 조언이 있다면 댓글에 남겨주세요